# Import Package

In [41]:
import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
# import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math

import Lib_RWA as Lib_RWA

del sys.modules['Lib_RWA']  # Remove the module from sys.modules to force a reload

import Lib_RWA as Lib_RWA

In [42]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\georgez\AppData\Local\Temp\ipykernel_30780\3764690850.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [43]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [44]:
# snapshot_num = 202212 # 20250412 George commented out
# snapshot = str(snapshot_num) # 20250412 George commented out
# snapshot_date = '2022-12-31' # 20250412 George commented out

snapshot_num = 202412 # 20250412 George added
snapshot = str(snapshot_num) # 20250412 George added
snapshot_date = '2024-12-31' # 20250412 George added


#CAR 2023 Ch5 prescribed
correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023


# Set Folders

In [45]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added

if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added

# Load PD Data

In [46]:
# Read PD Data

new_pd_data = pd.read_parquet(input_dir+ '\Concentra_PD_Scoring_2020-12-31_to_2024-12-31_wInsurer (from Francesca 20250411).parquet')  # 20250406 George added


# print columns in alphabetical order in the new_pd_data dataframe #20250402 George added

print('**************Before Renaming and add columns**************') #20250402 George added

print(sorted(new_pd_data.columns)) #20250402 George added





**************Before Renaming and add columns**************
['AIRB_scope_ind', 'Appraisal_Date_v2', 'Appraisal_Value_v2', 'Appraisal_YYYYMM', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Balance_Total', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_Orig', 'Beacon_Avg_App_CoApp_WOE', 'Beacon_Score_Current', 'Brookfield_Dwelling', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Concentra_Legacy_Ind', 'Concentra_ind', 'Current_Principal_Balance_CB', 'Days_in_Arrears', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Foreclosure', 'Funded_Date', 'Inception_date', 'Insured_Ind', 'Insurer', 'IsREO', 'Loan_Number', 'MRS_Bin_PD', 'Metro_Region_BF_FMT', 'Mortgage_Number', 'Nb_entities_per_loan', 'Occupancy_Status', 'Original_Loan_Amount', 'Original_Loan_to_Value', 'PD_Post_MOC', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Ad

In [ ]:
# Group by MRS_Bin, count distinct Mortgage_Number and min and max of PD_Post_MOC #20250402 George added
print('*************** Checking Deault Loans*****************') #20250402 George added
print(new_pd_data.groupby(['MRS_Bin_PD'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

In [ ]:
# # Remove Defaults


# # Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

# new_pd_data = new_pd_data.loc[(new_pd_data['PD_Post_MOC'] > 0) & (new_pd_data['PD_Post_MOC'] < 1)] #20250402 George added

# # Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

# print('***************After Removing Deault Loans*****************') #20250402 George added

# print(new_pd_data.groupby(['MRS_Bin_PD'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

***************Before Removing Deault Loans*****************
  MRS_Bin_PD Mortgage_Number PD_Post_MOC          
                     nunique         min       max
0        1.0           23096    0.002228  0.002228
1        2.0           15190    0.003622  0.003622
2        3.0            8616    0.007377  0.007377
3        4.0            5609    0.011499  0.011499
4        5.0            4817    0.019067  0.019067
5        6.0            4892    0.034554  0.034554
6        7.0            1650    0.107857  0.107857
7        8.0            1140    0.629118  0.629118
8        9.0             474    1.000000  1.000000
***************After Removing Deault Loans*****************
  MRS_Bin_PD Mortgage_Number PD_Post_MOC          
                     nunique         min       max
0        1.0           23096    0.002228  0.002228
1        2.0           15190    0.003622  0.003622
2        3.0            8616    0.007377  0.007377
3        4.0            5609    0.011499  0.011499
4        5.0

In [48]:
# print shape of new_pd_data #20250402 George added

print('***************Shape of new_pd_data*****************') #20250402 George added

print(new_pd_data.shape) #20250402 George added


# count values of islua_loan, loan_type, product_type and source_name

print('***************Count of islua_loan, loan_type, product_type and source_name*****************') #20250402 George added

print(new_pd_data.groupby(['islua_loan','loan_type', 'product_type', 'source_name'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique})) #20250402 George added

***************Shape of new_pd_data*****************
(1073510, 67)
***************Count of islua_loan, loan_type, product_type and source_name*****************
   islua_loan    loan_type product_type              source_name  Mortgage_Number
0       False  residential        alt-a                     cmls              815
1       False  residential        alt-a  community trust company              223
2       False  residential        alt-a           first national             8555
3       False  residential        alt-a                     mcap             3057
4       False  residential        prime      energy credit union               59
5       False  residential        prime           first national             7713
6       False  residential        prime   frontline credit union               42
7       False  residential        prime                     mcap            15117
8       False  residential        prime        pace credit union               63


In [49]:
# Exclude Loans
#  If islua_loan!=1 AND loan_type = 'residential' AND Balance_CB > 0 AND product_type NOT IN ('heloc','bridge financing') AND source_name IN ('mcap','first national','cmls') THEN exclude_ind = 0, else exclude_ind = 1 #20250402 George added

new_pd_data['exclude_ind'] = np.where(
    (new_pd_data['islua_loan'] != 1) &
    (new_pd_data['loan_type'] == 'residential') &
    (new_pd_data['Current_Principal_Balance_CB'] > 0) &
    (~new_pd_data['product_type'].isin(['heloc', 'bridge financing'])) &
    (new_pd_data['source_name'].isin(['mcap', 'first national', 'cmls'])),
    0,
    1
) #20250402 George added

# Group by exclude_ind, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

print('***************Count of exclude_ind*****************') #20250402 George added

print(new_pd_data.groupby(['exclude_ind'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

***************Count of exclude_ind*****************
  exclude_ind Mortgage_Number PD_Post_MOC          
                      nunique         min       max
0           0           35243    0.002228  0.629118
1           1             698    0.002228  0.629118


In [50]:
# Rename columns:  SL_Date -> Snapshot_Date, AIRB_PD_Risk_Rating -> MRS_Bin, Apprisal_Value -> Appraisal_Value_v2, Appraisla_Date -> Appraisal_Date_v2, occupancy_status -> Occupancy_Status, Balance_CB to Current_Principal_Balance_CB, 

# new_pd_data.rename(columns={'SL_Date':'Snapshot_Date', 'AIRB_PD_Risk_Rating':'MRS_Bin', 'Appraisal_Value':'Appraisal_Value_v2', 'Appraisal_Date':'Appraisal_Date_v2', 'occupancy_status':'Occupancy_Status', 'Balance_CB':'Current_Principal_Balance_CB'}, inplace=True)

# If columns PD_Post_MOC_Pre_Adj,def_90dpd_utp_filldown, exclude_ind  are missing, add them as NaN

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data.columns: # 20250412 George added

    new_pd_data['PD_Post_MOC_Pre_Adj'] = np.nan  # 20250412 George added

if 'def_90dpd_utp_filldown' not in new_pd_data.columns: # 20250412 George added

    new_pd_data['def_90dpd_utp_filldown'] = np.nan  # 20250412 George added

if 'exclude_ind' not in new_pd_data.columns: # 20250412 George added

    new_pd_data['exclude_ind'] = 0  # 20250412 George added

# print columns in alphabetical order in the new_pd_data dataframe #20250402 George added
print('**************AFter Renaming and add columns**************') #20250402 George added

print(sorted(new_pd_data.columns)) #20250402 George added

**************AFter Renaming and add columns**************
['AIRB_scope_ind', 'Appraisal_Date_v2', 'Appraisal_Value_v2', 'Appraisal_YYYYMM', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Balance_Total', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_Orig', 'Beacon_Avg_App_CoApp_WOE', 'Beacon_Score_Current', 'Brookfield_Dwelling', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Concentra_Legacy_Ind', 'Concentra_ind', 'Current_Principal_Balance_CB', 'Days_in_Arrears', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Foreclosure', 'Funded_Date', 'Inception_date', 'Insured_Ind', 'Insurer', 'IsREO', 'Loan_Number', 'MRS_Bin_PD', 'Metro_Region_BF_FMT', 'Mortgage_Number', 'Nb_entities_per_loan', 'Occupancy_Status', 'Original_Loan_Amount', 'Original_Loan_to_Value', 'PD_Post_MOC', 'PD_Post_MOC_Pre_Adj', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_De

In [51]:
# Filter snapshot, exclusion and keep columns
new_pd_data_red = new_pd_data.loc[(new_pd_data['Snapshot_Date'] == snapshot_date)&(new_pd_data['PD_Post_MOC'].notna())&(new_pd_data['exclude_ind']==0),['Mortgage_Number','Snapshot_Date','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD','Insured_Ind','Appraisal_Value_v2',
                                                             'Combo_LTV_Insured_Ind',  'Combo_Province_Metro_Override_WOE', 'def_90dpd_utp_filldown', 'Concentra_Legacy_Ind', 'Occupancy_Status','Original_Loan_to_Value', 'FSA','Appraisal_Date_v2','Current_Principal_Balance_CB','Insurer','Original_Loan_Amount','Original_Loan_to_Value']].rename(columns={'MRS_Bin':'MRS_Bin_PD'})

# change Mortgage_Number to string because later maturity data has Mortgage_Number as string (some Mortgage number in maturity starts with C as a string)

new_pd_data_red['Mortgage_Number'] = new_pd_data_red['Mortgage_Number'].astype(str) # 20250412 George added

In [52]:
new_pd_data_red.groupby(['PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD'],dropna=False).size()

PD_Pre_MOC  PD_Post_MOC_Pre_Adj  PD_Post_MOC  MRS_Bin_PD
0.001538    NaN                  0.002228     1.0           10644
0.002667    NaN                  0.003622     2.0            3720
0.005465    NaN                  0.007377     3.0            1407
0.008601    NaN                  0.011499     4.0             784
0.014295    NaN                  0.019067     5.0             367
0.026508    NaN                  0.034554     6.0             307
0.081517    NaN                  0.107857     7.0             322
0.477554    NaN                  0.629118     8.0             104
dtype: int64

In [53]:
new_pd_data_red.Concentra_Legacy_Ind.value_counts()

0.0    15916
1.0     1739
Name: Concentra_Legacy_Ind, dtype: int64

In [54]:
# get Maturity - old Data strategy - Francesca did not use this code for Remaining_Term (instead LGD data has that column)
# import pyodbc
# server = 'EQSQLT01\RISK'  #Note that python is case sensitive, the server name has to be capital as they actually are
# database = 'Concentra'
# conn =pyodbc.connect(f'Driver={{SQL Server}}; Server={server};Database={database}')
# sql_query_loan = f'''
#     SELECT 
#         Mortgage_Number,
#         Remaining_Term
#        ,[Maturity_Date]
#        ,[Days_in_Arrears]
#     FROM 
#         Concentra.dbo.vw_CN_SFR_Loan_Dataset
#     WHERE
#         CONVERT(DATETIME, EOMONTH(SL_Date))='{snapshot_date}' and
#         IsLUA_Loan != 1 AND
#         Loan_Type = 'Residential' AND
#         Current_Principal_Balance_CB > 0 AND
#         Product_Type NOT IN ('HELOC','Bridge Financing') AND
#         Source_Name in ('MCAP','First National','CMLS')
# '''

# maturity = pd.read_sql_query(sql_query_loan,conn,parse_dates=['Maturity_Date'])

In [55]:
# maturity.Mortgage_Number=maturity.Mortgage_Number.str.lstrip('0')  # Again, the remaining_term is not used as this is old data strategy

# Load LGD Data

In [56]:
# new_lgd_data = pd.read_pickle(r"C:\Users\jasonz\OneDrive - Equitable Bank\Fit_for_Use Development\git_repos_MD\abhignanp\tempResults\Concentra_LGD_AllScored_Data.pkl") # 20250412 George commented out

new_lgd_data = pd.read_parquet(input_dir + '\Concentra_LGD_Scoring_2020-12-31_to_2024-12-31 (from Francesca 20250411).parquet')  # 20250406 George added


# change Mortgage_Number to string because later maturity data has Mortgage_Number as string (some Mortgage number in maturity starts with C as a string)

new_lgd_data['Mortgage_Number'] = new_lgd_data['Mortgage_Number'].astype(str) # 20250412 George added

new_lgd_data.loc[new_lgd_data['Remaining_Term']<0,'Remaining_Term']=0  #pasting Francesca's code to clean up Remaining_Term # 20250411 George added

In [57]:
# print min and max of Snapshot_Date in new_lgd_data #20250411 George added

print('***************Snapshot_Date in new_lgd_data*****************') #20250411 George added

print(new_lgd_data['Snapshot_Date'].min()) #20250411 George added

print(new_lgd_data['Snapshot_Date'].max()) #20250411 George added

***************Snapshot_Date in new_lgd_data*****************
2020-12-31 00:00:00
2024-12-31 00:00:00


In [58]:
# Filter LGD Data to have Snapshot_Date = snapshot_date

new_lgd_data_red = new_lgd_data.loc[new_lgd_data['Snapshot_Date'] == snapshot_date]

# count values by Snapshot_Date

print('***************Count of Snapshot_Date in new_lgd_data*****************') 

print(new_lgd_data_red.groupby(['Snapshot_Date'], as_index=False).agg({'Mortgage_Number': pd.Series.nunique})) 

***************Count of Snapshot_Date in new_lgd_data*****************
  Snapshot_Date  Mortgage_Number
0    2024-12-31            17847


# Merge PD and LGD Data

In [ ]:
rwa_data=new_pd_data_red.merge(new_lgd_data_red[['Mortgage_Number','MRS_Bin','Pred_LGD','Mapped_LGD','Final_LGD','Remaining_Term']].rename(columns={'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}), on ='Mortgage_Number', how='outer')

In [60]:
rwa_data['Remaining_Term'].describe()

count    17655.000000
mean        17.113566
std         13.090619
min          0.000000
25%          7.000000
50%         13.000000
75%         24.000000
max         60.000000
Name: Remaining_Term, dtype: float64

In [61]:
rwa_data.Insurer.value_counts(dropna= False) # 20250403 FRAN added to verify below is correct - modified Cmhc: CMHC

Conventional       6633
Canada Guaranty    5835
CMHC               5187
Name: Insurer, dtype: int64

# Map Insurer to Insured_class

In [62]:
insured_class = {'Sagen':'Sagen', 'Conventional':'Uninsured', 'Canada Guaranty':'CG', 'CMHC Portfolio':'CMHC',
       'Cmhc':'CMHC', 'Sagen Portfolio':'Sagen', 'Canada Guaranty Portfolio':'CG'}

# if Insurer is one of the keys in the dictionary, then assign the value to the Insured_class column, otherwise assign the original value of Insurer to the Insured_class column

rwa_data['Insured_class'] = rwa_data['Insurer'].apply(lambda x: insured_class[x] if x in insured_class else x) # 20250411 George added

# rwa_data['Insured_class']= rwa_data['Insurer'].map(insured_class) # 20250411 George commented out because CMHC was not in the dictionary and changed to missing if not in the dictionary

# assign missing values in Insured_class to Uninsured # 20250411 George added

rwa_data['Insured_class'].fillna('Uninsured', inplace=True) # 20250411 George added


# value counts Insured_class

print('***************Count of Insured_class in rwa_data*****************') 

print(rwa_data['Insured_class'].value_counts(dropna= False)) 

***************Count of Insured_class in rwa_data*****************
Uninsured    6633
CG           5835
CMHC         5187
Name: Insured_class, dtype: int64


In [63]:

# corr_uninsured
rwa_data['corr_uninsured'] =rwa_data['Occupancy_Status'].map(lambda x:0.15 if x in ['owner occupied','Owner Occupied','Owner_Occupied','owner_occupied'] else 0.22) # 20250412 George added

In [64]:
rwa_data.columns.values

array(['Mortgage_Number', 'Snapshot_Date', 'PD_Pre_MOC',
       'PD_Post_MOC_Pre_Adj', 'PD_Post_MOC', 'MRS_Bin_PD', 'Insured_Ind',
       'Appraisal_Value_v2', 'Combo_LTV_Insured_Ind',
       'Combo_Province_Metro_Override_WOE', 'def_90dpd_utp_filldown',
       'Concentra_Legacy_Ind', 'Occupancy_Status',
       'Original_Loan_to_Value', 'FSA', 'Appraisal_Date_v2',
       'Current_Principal_Balance_CB', 'Insurer', 'Original_Loan_Amount',
       'Original_Loan_to_Value', 'MRS_Bin_LGD', 'Pred_LGD', 'Mapped_LGD',
       'Model_LGD', 'Remaining_Term', 'Insured_class', 'corr_uninsured'],
      dtype=object)

# Add Other columns 

In [65]:

# if LGD_DT_Adjusted is not in the columns, add it as NaN

if 'LGD_DT_Adjusted' not in rwa_data.columns: # 20250412 George added
    rwa_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

rwa_data['Segment_Avg_LGD'] = rwa_data.loc[:,'Mapped_LGD']
rwa_data['Base_Line_LGD'] =rwa_data.loc[:,'Mapped_LGD']
rwa_data['Advance_Amount'] = rwa_data.loc[:,'Original_Loan_Amount']
rwa_data['Years_to_maturity'] = rwa_data.loc[:,'Remaining_Term']/12
rwa_data['EAD'] = rwa_data.loc[:,'Current_Principal_Balance_CB']

# RWA with 0 Addon

In [66]:
# Force Addon to be 0

rwa_raw_data_5 = rwa_data.copy()
rwa_raw_data_5['AddOn']=0


In [67]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_5, lgd_gen_floor, CMHC_lgd, CMHC_pd)



0
1
2


In [68]:
# print columns of res_data in alphabetical order

print('***************Columns of res_data in alphabetical order*****************') 

print(sorted(res_data.columns)) 

***************Columns of res_data in alphabetical order*****************
['AddOn', 'Advance_Amount', 'Appraisal_Date_v2', 'Appraisal_Value_v2', 'Base_Line_LGD', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_Province_Metro_Override_WOE', 'Concentra_Legacy_Ind', 'Current_Principal_Balance_CB', 'DLGD_floor', 'EAD', 'EAD_DLGD_Modified', 'EAD_Post_CRM', 'FSA', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'Insurer', 'LGD_Before_Floors', 'LGD_DT_Adjusted', 'MRS_Bin_LGD', 'MRS_Bin_PD', 'Mapped_LGD', 'Maturity_adj', 'Model_LGD_Modified', 'Mortgage_Number', 'OSFI_LGD_floor', 'Occupancy_Status', 'Original_Loan_Amount', 'Original_Loan_to_Value', 'Original_Loan_to_Value', 'Pre_final_LGD', 'Pred_LGD', 'RWA_new', 'Remaining_Term', 'Segment_Avg_LGD', 'Snapshot_Date', 'Years_to_maturity', 'corr_insured', 'corr_uninsured', 'deductible_amount', 'def_90dpd_utp_filldown', 'insured_PMI_ratio', 'risk_weight_insured', 'risk_weight_uninsured']


In [69]:
# Save Data to pickle file

res_data.to_pickle(Output_dir + '\\' + 'concentra_rwa_addon_zero.pkl') 

# Borrow Add on Impact from EQB

In [70]:
# We need to add a cross table of lgd_bins and PD bins for rwa with add-on. However, we don't have account level addon info. therefore we take the impact from EQB at the Combo_LTV_Insured_Ind X Combo_Province_Metro_Override_WOE level

# addon_impact =pd.read_excel(r"C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\addon_impact.xlsx")  # 20250406 George commented out Francesca's code

addon_impact =pd.read_excel(Output_dir+r"\addon_impact.xlsx") # 20250406 George added

rwa_raw_data = res_data.copy() # 20250411 George added because a new res_data is created in the above function

In [71]:
# 20250403 - Francesca Add on already has mapped values !
addon_impact.Combo_Province_Metro_Override_WOE.value_counts()

-0.389936    7
 0.165650    7
-0.161658    6
 0.253031    6
Name: Combo_Province_Metro_Override_WOE, dtype: int64

In [72]:
addon_impact.Combo_Province_Metro_Override_WOE.fillna(0, inplace=True)
addon_impact.Combo_Province_Metro_Override_WOE =addon_impact.Combo_Province_Metro_Override_WOE.round(8)

In [73]:
addon_impact.rename(columns={'impact':'addon_impact'},inplace=True)

In [74]:
#process to make sure the merge works;
rwa_raw_data.Combo_Province_Metro_Override_WOE=rwa_raw_data.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_Province_Metro_Override_WOE = addon_impact.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_LTV_Insured_Ind=addon_impact.Combo_LTV_Insured_Ind.fillna('')

In [75]:
rwa_raw_data_addon=rwa_raw_data.merge(addon_impact[['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE','addon_impact']], on=['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],how='left')

In [76]:
rwa_raw_data_addon.addon_impact = rwa_raw_data_addon.addon_impact.fillna(1)

In [77]:
rwa_raw_data_addon['RWA_AddOn']=rwa_raw_data_addon['RWA_new']*rwa_raw_data_addon['addon_impact']

# Save Data

In [78]:

# Save Data to Pickle

with open( Output_dir +r'\concentra_rwa_addon.pkl', 'wb') as file: # 20250406 George added

    # A new file will be created 
    pickle.dump(rwa_raw_data_addon, file) 

In [79]:
# #Save an excel copy

# rwa_raw_data_addon.to_excel(Output_dir + r'\concentra_rwa_addon.xlsx')  #20250406 George added

In [80]:
# print shape of rwa_raw_data_addon

print('***************Shape of rwa_raw_data_addon*****************')

print(rwa_raw_data_addon.shape)

***************Shape of rwa_raw_data_addon*****************
(17655, 48)
